# Raw time of flight

In [1]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import random
from scipy import signal
from IPython.display import HTML, display,clear_output
import ipywidgets as wg


In [2]:
def create_fake_tof(length= 1000, peaks = 20):
    time_axis = np.linspace(0,length/2, length)
    tof_array = np.random.random(length)
    peak_pos = random.sample(range(5,length),peaks)
    peak_height = np.random.random(peaks)*10
    tof_array[peak_pos] = peak_height
    return time_axis, tof_array

def plot_trace(axis, tof_array, fitted_peaks, x_label = 'time [u secs]'):
    plt.figure(figsize=(9, 8))
    plt.plot(axis, tof_array)
    plt.plot(axis[fitted_peaks[0]],fitted_peaks[1]['peak_heights'],'ro')
    plt.title("Average TOF")
    plt.ylabel('Intensity [a.u.]')
    plt.xlabel(x_label)
    plt.show();
    
def print_peaks(axis, fitted_peaks, decimals = 3):
    data = [(axis[fitted_peaks[0]][index],fitted_peaks[1]['peak_heights'][index]) for index in range(len(axis[fitted_peaks[0]]))]
    data = np.around(data, decimals = decimals)
    display(HTML('<h3>Pos. Height</h3>'))
    display(HTML(
        '<table><tr>{}</tr></table>'.format(
            '</tr><tr>'.join(
                '<td>{}</td>'.format('</td><td>'.join(str(_) for _ in row)) for row in data)
        )
    ))
    
def plot_trace_and_print_peak_table(axis, tof_array, fitted_peaks, x_label = 'time [u secs]'):
    plot_trace(axis, tof_array, fitted_peaks, 'time [u secs]')
    print_peaks(axis,fitted_peaks)
    

In [3]:
# tof_array = np.load('../scripts/average_tof.npy')

time_axis, tof_array = create_fake_tof()
fitted_peaks = signal.find_peaks(tof_array, height=2)

In [4]:
plot_trace_and_print_peak_table(time_axis, tof_array, fitted_peaks, 'time [u secs]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

12.012,2.818
63.564,7.136
66.567,3.778
91.592,3.299
93.594,3.212
111.612,4.134
112.613,8.363
154.154,4.856
180.681,6.188
257.257,8.218
272.773,8.492


# Mass calibration 

<font size="4">
$
\frac{m}{z} = a \ \tau^2  + b \\
a = \frac{\frac{m}{z}_1 - \frac{m}{z}_2}{\tau_1^2 - \tau_2^2} \quad \quad 
b = \frac{m}{z}_1 - \frac{\frac{m}{z}_1 - \frac{m}{z}_2}{\tau_1^2 - \tau_2^2} \tau_1^2
$
</font>

In [24]:
def mass_calibration(time_axis):
    m_over_z_ion_1 = ion_1_dropdown.value
    tof_ion_1 = ion_1_m_over_z.value
    m_over_z_ion_2 = ion_2_dropdown.value
    tof_ion_2 = ion_2_m_over_z.value
    a = (m_over_z_ion_1-m_over_z_ion_2)/(tof_ion_1**2-tof_ion_2**2)
    b = m_over_z_ion_1 - (m_over_z_ion_1-m_over_z_ion_2)/(tof_ion_1**2-tof_ion_2**2) * tof_ion_1**2
    return a * time_axis * time_axis + b

def on_button_clicked_mass_calibration(b):
    with output_mass_calibration:
        clear_output(True)
        plot_trace_and_print_peak_table(mass_calibration(time_axis), tof_array, fitted_peaks, x_label = 'time [u secs]')

In [25]:
dropdown_values = np.around(time_axis[fitted_peaks[0]], decimals=2)

ion_1_dropdown = wg.Dropdown(
    options=dropdown_values,
    value=dropdown_values[0],
    description='\tau TOF Ion 1:',
    disabled=False,
)

ion_2_dropdown = wg.Dropdown(
    options=dropdown_values,
    value=dropdown_values[1],
    description='TOF Ion 2:',
    disabled=False,
)

ion_1_m_over_z = wg.BoundedFloatText(
    value=5,
    min=1,
    max=500,
    step=0.01,
    description='m/z:',
    disabled=False
)

ion_2_m_over_z = wg.BoundedFloatText(
    value=10,
    min=1,
    max=500,
    step=0.01,
    description='m/z:',
    disabled=False
)

button_mass_calibration = wg.Button(description="Mass calibration")
output_mass_calibration = wg.Output()

left_box = wg.VBox([ion_1_dropdown,ion_2_dropdown])
right_box = wg.VBox([ion_1_m_over_z,ion_2_m_over_z])

display(wg.HBox([left_box, right_box, button_mass_calibration]), output_mass_calibration)

button_mass_calibration.on_click(on_button_clicked_mass_calibration)

Output()